# 13 - Medium Level Example - Modeling Modules with Glass


This journal shows how to add glass to the modules. It also shows how to sample irradiance on the surface of the glass, as well as on the surface of the module. Surface of the module is slightlyt less irradiance due to fresnel losses (a.k.a. Angle of Incidence losses (AOI))

Some examples of the module with glass we'll create:

![Bottom Reflection](../images_wiki/Journal13Pics/Glass_bottom_reflection.PNG)
![Side closeup](../images_wiki/Journal13Pics/Glass_side_closeup.PNG)
![Tiled view](../images_wiki/Journal13Pics/Glass_tilted_reflection.PNG)


In [1]:
import os
from bifacial_radiance import *
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


In [2]:
demo = RadianceObj('bifacial_example',str(testfolder))  
demo.setGround(0.30)  # This prints available materials.
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(epwfile) 
demo.gendaylit(8)  # Noon, June 17th (timepoint # 4020)\


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!
Saving file EPWs\epw_temp.csv, # points: 8760


'skies\\sky2_37.5_-77.33_06_17_13.rad'

### Modeling example with glass

In [12]:
module_type = 'Bi60' 

numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

torquetube = True
diameter = 0.15
tubetype = 'round'
material = 'Metal_Grey'
xgap = 0.1
ygap = 0
zgap = 0.05
numpanels = 1
axisofrotationTorqueTube = False
glass = True

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=module_type, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                xgap=xgap, ygap=ygap, zgap=zgap, numpanels=numpanels, 
                cellLevelModuleParams=cellLevelModuleParams, 
                axisofrotationTorqueTube=axisofrotationTorqueTube, glass=glass, z=0.0002)

sceneDict = {'tilt':25,'pitch':5.5,'hub_height':1.0,'azimuth':90, 'nMods': 20, 'nRows': 1, originx=0, originy=0} 
scene = demo.makeScene(module_type,sceneDict)
octfile = demo.makeOct(demo.getfilelist())  


Module Name: Bi60
REWRITING pre-existing module file. 
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Bi60 successfully created
Created bifacial_example.oct


In [15]:
sceneDict = {'tilt':25,'pitch':5.5,'hub_height':1.0,'azimuth':90, 'nMods': 20, 'nRows': 10, 'originx':0, 'originy':0} 
scene = demo.makeScene(module_type,sceneDict)
octfile = demo.makeOct(demo.getfilelist())  

Created bifacial_example.oct


## Advanced Rendering:

The images at the beginning of the journal can be made pretty with advanced rendering. This is the workflow:

    rvu -> rpict -> pcond -> pfilt -> ra_tiff -> convert

In detail:
##### 1.	Use rvu to view the oct file. 

    rvu 1axis_07_01_08.oct 

Use aim and origin to move around, zoom in/out, etc. Save a view file with view render.

##### 2.	Run rpict to render the image to hdr. 
This is the time consuming step. It takes between 1 and 3 hours depending on how complex the geometry is.

    rpict -x 4800 -y 4800 -i -ps 1 -dp 530 -ar 964 -ds 0.016 -dj 1 -dt 0.03 -dc 0.9 -dr 5 -st 0.12 -ab 5 -aa 0.11 -ad 5800 -as 5800 -av 25 25 25 -lr 14 -lw 0.0002 -vf render.vf bifacial_example.oct > render.hdr


##### 3.	Run pcond to mimic human visual response

    pcond -h render.hdr > render.pcond.hdr


##### 4.	Resize and adjust exposure with pfilt

    pfilt -e +0.2 -x /4 -y /4 render.pcond.hdr > render.pcond.pfilt.hdr


##### 5.	Convert hdr to tif

    ra_tiff render.pcond.pfilt.hdr render.tif


##### 6.	Convert tif to png with imagemagick convert utility

    convert render.tif render.png


##### 7.	Annotate the image with convert

    convert render.png -fill black -gravity South -annotate +0+5 'Created with NREL bifacial_radiance https://github.com/NREL/bifacial_radiance' render_annotated.png


In [8]:
analysis = AnalysisObj(octfile, demo.basename)


## Scanning Outside of the module, the surface of the glass

In [9]:
frontscan, backscan = analysis.moduleAnalysis(scene)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_bifacial_example.csv')

Linescan in process: bifacial_example_Front
Linescan in process: bifacial_example_Back
Saved: results\irr_bifacial_example.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,8.402972e-17,-0.686155,0.521014,0.519212,a0.0.a2.1.0.cellPVmodule.6457,a0.0.a2.1.0.cellPVmodule.2310,508.529533,73.868537,0.145259
1,6.304336e-17,-0.514788,0.641006,0.639204,a0.0.a2.2.0.cellPVmodule.6457,a0.0.a2.2.0.cellPVmodule.2310,508.971733,73.021703,0.143469
2,4.205700e-17,-0.343421,0.760999,0.759197,a0.0.a2.3.0.cellPVmodule.6457,a0.0.a2.3.0.cellPVmodule.2310,509.416100,75.392390,0.147997
3,2.107065e-17,-0.172055,0.880991,0.879189,a0.0.a2.4.0.cellPVmodule.6457,a0.0.a2.4.0.cellPVmodule.2310,508.580900,73.499067,0.144518
4,8.429143e-20,-0.000688,1.000983,0.999181,a0.0.a0.Bi60_Glass.2310,a0.0.a0.Bi60_Glass.6457,310.609033,209.151200,0.673356
5,-2.090207e-17,0.170678,1.120975,1.119173,a0.0.a2.7.0.cellPVmodule.6457,a0.0.a2.7.0.cellPVmodule.2310,509.044100,90.007653,0.176817
6,-4.188842e-17,0.342045,1.240967,1.239165,a0.0.a2.8.0.cellPVmodule.6457,a0.0.a2.8.0.cellPVmodule.2310,508.808333,93.551353,0.183863
7,-6.287478e-17,0.513412,1.360960,1.359157,a0.0.a2.9.0.cellPVmodule.6457,a0.0.a2.9.0.cellPVmodule.2310,508.999567,93.880210,0.184440
8,-8.386114e-17,0.684778,1.480952,1.479150,a0.0.a2.10.0.cellPVmodule.6457,a0.0.a2.10.0.cellPVmodule.2310,509.193167,96.795070,0.190095


## Scanning Inside of the module, the surface of the cells

In [10]:
frontscan, backscan = analysis.moduleAnalysis(scene, frontsurfaceoffset=0.02, backsurfaceoffset = 0.02)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_bifacial_example.csv')

Linescan in process: bifacial_example_Front
Linescan in process: bifacial_example_Back
Saved: results\irr_bifacial_example.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,8.536433e-17,-0.697053,0.536578,0.503648,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,508.191000,72.608870,0.142877
1,6.437798e-17,-0.525686,0.656570,0.623640,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,508.580967,73.231090,0.143991
2,4.339162e-17,-0.354320,0.776563,0.743633,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,508.973233,75.609833,0.148553
3,2.240526e-17,-0.182953,0.896555,0.863625,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,509.056333,71.672763,0.140795
4,1.418906e-18,-0.011586,1.016547,0.983617,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,96.115980,209.151200,2.176007
5,-1.956745e-17,0.159780,1.136539,1.103609,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,508.812267,88.169193,0.173284
6,-4.055381e-17,0.331147,1.256531,1.223601,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,509.010700,90.714453,0.178217
7,-6.154016e-17,0.502514,1.376523,1.343594,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,509.211233,94.724320,0.186021
8,-8.252652e-17,0.673880,1.496516,1.463586,a0.0.a0.Bi60_Glass.6457,a0.0.a0.Bi60_Glass.2310,509.411833,96.086757,0.188623
